In [ ]:
#| default_exp datastructure.fixture

In [ ]:
from IPython.core.debugger import set_trace

%load_ext autoreload
%autoreload 2

#  Fixtures
> Data structure required to extract soccer fixtures.

In [ ]:
#| export

import pandas as pd
import mongoengine
import datetime
import logging
from typing import List
from mongoengine.fields import ListField
from soccer_prediction_data.config.localconfig import CONFIG

## Venue and Officials classes

The Venue and officials classes contain the necassary information about the venue and officials reported on the game.

In [ ]:
#| export


class Venue(mongoengine.EmbeddedDocument):
    "Stores main information about the reported venue on the game information."
    # Venue Ids.
    venue_id = mongoengine.StringField(db_field="VenueId", required=False)
    # Venue name.
    venue_name = mongoengine.StringField(db_field="VenueName", required=False)


class Officials(mongoengine.EmbeddedDocument):
    "Stores main information about officials reported on the game information."
    # Venue Ids.
    referee_id = mongoengine.IntField(db_field="refereeId", required=False)
    # Venue name.
    referee_name = mongoengine.StringField(db_field="refereeName", required=False)

## Games

These features are computed at the game level and contains:

+ **Metadata**: general information about game date, teams identities, competition, season, duration, officials...
+ **Stadium** information: Venue name and Id.
+ **Full Time Score**: Scored goals.

In [ ]:
#| export


class Fixture(mongoengine.Document):
    "Store the game-features for a given game identified by its id"

    # Game-id.
    game_id = mongoengine.IntField(db_field="gameId", required=True)

    # Game-date/time.
    game_date = mongoengine.DateTimeField(db_field="gameDate", required=True)
    season_id = mongoengine.IntField(db_field="seasonId", required=True)
    season_name = mongoengine.StringField(db_field="seasonName", required=True)
    match_day = mongoengine.IntField(db_field="matchday", required=True)
    period = mongoengine.StringField(db_field="period", required=True)
    duration = mongoengine.IntField(db_field="duration", required=False)
    last_modified = mongoengine.DateTimeField(db_field="lastModified", required=False)

    # Competition.
    competition_id = mongoengine.IntField(db_field="competitionId", required=True)
    competition_name = mongoengine.StringField(
        db_field="competitionName", required=True
    )

    # Home team.
    home_team_id = mongoengine.IntField(db_field="homeTeamId", required=True)
    home_team_name = mongoengine.StringField(db_field="homeTeamName", required=True)

    # Away team.
    away_team_id = mongoengine.IntField(db_field="awayTeamId", required=True)
    away_team_name = mongoengine.StringField(db_field="awayTeamName", required=True)

    # Stadium.
    stats = mongoengine.EmbeddedDocumentListField(Venue)

    # Officials.
    officials = mongoengine.EmbeddedDocumentListField(Officials)

    # Final score.
    full_time_score = ListField(db_field="fullTimeScore", required=False)

    # Metadata.
    events_available = mongoengine.BooleanField(
        db_field="eventsAvailable", required=False
    )

    meta = {
        "db_alias": "inStat",
        "collection": "Fixture",
    }

    @classmethod
    def get_all_games(
        cls,
        limit: int = None,  # Number of rows.
    ):  # List of Fixture output.
        "Extract all games."
        return cls.objects().order_by("gameDate").limit(limit)

    @classmethod
    def get_game(
        cls,
        game_id: int,  # Instat game identifier.
    ):  # Fixture output.
        "Extract a single game by its gameId."
        return cls.objects(game_id=game_id).first()

    @classmethod
    def get_games_by_competition(
        cls,
        competition_ids: List[int],  # List of competition identifiers.
        limit: int = None,  # Number of rows.
    ):  # List of Fixture output.
        "Extract all games of given competitions."
        return (
            cls.objects(competition_id__in=competition_ids)
            .order_by("gameDate")
            .limit(limit)
        )

In [ ]:
from soccer_prediction_data.config.mongo import mongo_init

In [ ]:
# Initialise connections.
mongo_init("prod_atlas")
# GameId.
game_id = 1199088
# Extract game info.
game_info = Fixture.get_game(game_id)
{x: game_info[x] for x in game_info}

{'id': 1199088,
 'game_id': 1199088,
 'game_date': datetime.datetime(2018, 7, 29, 5, 6),
 'season_id': 2018,
 'season_name': '2018-2019',
 'match_day': 2,
 'period': 'Breakdown completed',
 'duration': 96,
 'last_modified': datetime.datetime(2022, 11, 3, 5, 7, 3, 611000),
 'competition_id': 108,
 'competition_name': 'Mexico. Liga MX',
 'home_team_id': 1889,
 'home_team_name': 'Guadalajara',
 'away_team_id': 1059,
 'away_team_name': 'Cruz Azul',
 'stats': [<Venue: Venue object>],
 'officials': [<Officials: Officials object>],
 'full_time_score': [0, 1],
 'events_available': True}

In [ ]:
#| hide

import nbdev

nbdev.nbdev_export()